## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
import pandas as pd
import tensorflow as tf

In [0]:
data = pd.read_csv('/content/drive/My Drive/ColabNotebooks/prices.csv')

### Check all columns in the dataset

In [67]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'], axis =1 )

In [69]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data.head(1000)

In [71]:
data.shape

(1000, 5)

In [72]:
data.columns

Index(['open', 'close', 'low', 'high', 'volume'], dtype='object')

In [0]:
data['volume'] = data['volume']/1000000

In [74]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X =  data.drop("volume", axis=1)
y =  data.pop("volume")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X, norm='l2')

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')
x_n = tf.nn.l2_normalize(x,1)
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100

In [83]:
for epoch in range(training_epochs):
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  225.87958
Training loss at step:  5  is  203.89145
Training loss at step:  10  is  197.76749
Training loss at step:  15  is  196.0619
Training loss at step:  20  is  195.58687
Training loss at step:  25  is  195.45459
Training loss at step:  30  is  195.41777
Training loss at step:  35  is  195.40747
Training loss at step:  40  is  195.40462
Training loss at step:  45  is  195.40384
Training loss at step:  50  is  195.40361
Training loss at step:  55  is  195.40355
Training loss at step:  60  is  195.40353
Training loss at step:  65  is  195.40353
Training loss at step:  70  is  195.40353
Training loss at step:  75  is  195.40353
Training loss at step:  80  is  195.40353
Training loss at step:  85  is  195.40353
Training loss at step:  90  is  195.40353
Training loss at step:  95  is  195.40353


### Get the shapes and values of W and b

In [84]:
W.shape

TensorShape([Dimension(4), Dimension(1)])

In [85]:
b.shape

TensorShape([Dimension(1)])

### Model Prediction on 1st Examples in Test Dataset

In [86]:
test_loss = sess.run([loss],feed_dict={x:X_test, y_:y_test})
print ('Training loss at step:is ', test_loss)

Training loss at step:is  [241.01949]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
keras_data = pd.read_csv('/content/drive/My Drive/ColabNotebooks/11_Iris.csv')

In [89]:
keras_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
keras_data = pd.get_dummies(keras_data, prefix=['Species'], columns=['Species'])

In [92]:
keras_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [99]:
keras_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [98]:
type(data)

pandas.core.frame.DataFrame

### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
X =  keras_data.drop(["Species_Iris-setosa","Species_Iris-versicolor","Species_Iris-virginica","Id"], axis=1)
y =  keras_data[["Species_Iris-setosa","Species_Iris-versicolor","Species_Iris-virginica"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [108]:
from keras.models import Sequential
from keras.layers import Dense

# create model
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


W0721 11:16:31.101130 139815947892608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 11:16:31.108166 139815947892608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 11:16:31.115797 139815947892608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0721 11:16:31.168733 139815947892608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 11:16:31.195473 139815947892608 deprecation_wrappe

### Model Training 

In [110]:
model.fit(X_train,y_train , epochs=100, batch_size=10)

W0721 11:17:37.409340 139815947892608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0721 11:17:37.475849 139815947892608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
105/105 [==============================] - 0s 3ms/step - loss: 1.5848 - acc: 0.3429
Epoch 2/100
105/105 [==============================] - 0s 435us/step - loss: 1.1925 - acc: 0.4667
Epoch 3/100
105/105 [==============================] - 0s 419us/step - loss: 1.0854 - acc: 0.6952
Epoch 4/100
105/105 [==============================] - 0s 431us/step - loss: 1.0465 - acc: 0.6952
Epoch 5/100
105/105 [==============================] - 0s 416us/step - loss: 1.0250 - acc: 0.6952
Epoch 6/100
105/105 [==============================] - 0s 441us/step - loss: 1.0044 - acc: 0.6952
Epoch 7/100
105/105 [==============================] - 0s 427us/step - loss: 0.9830 - acc: 0.6952
Epoch 8/100
105/105 [==============================] - 0s 425us/step - loss: 0.9600 - acc: 0.6952
Epoch 9/100
105/105 [==============================] - 0s 409us/step - loss: 0.9385 - acc: 0.6952
Epoch 10/100
105/105 [==============================] - 0s 401us/step - loss: 0.9128 - acc: 0.6952
Epoch 11/100
105/105 

### Model Prediction

In [112]:
scores = model.evaluate(X_train, y_train)
scores[1]*100

105/105 [==============================] - 0s 228us/step


96.19047624724251

### Save the Model

In [0]:
model.save('/content/drive/My Drive/ColabNotebooks/model.hdf')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?